In [ ]:

# -------------------------------------------------------------------
# VALIDATION FINALE U=12, N=2000, PRÉCISION=100
# -------------------------------------------------------------------
import mpmath as mp
import numpy as np
import scipy.linalg as la
from scipy.special import roots_legendre
from scipy.stats import kstest
import time
import json
import matplotlib.pyplot as plt

class RiemannOperatorFinal:
    """Opérateur de Riemann avec paramètres optimaux pour la preuve finale"""

    def __init__(self, U=12, N=2000, precision=100):
        self.U = U
        self.N = N
        self.precision = precision
        mp.dps = precision

        # Discrétisation avec quadrature de Gauss-Legendre
        nodes, weights = roots_legendre(N)
        self.nodes = 0.5 * (nodes + 1) * (2 * U) - U
        self.weights = weights * U
        self.x = np.exp(self.nodes)

    def kernel_function(self, u, v):
        """Noyau de l'opérateur avec optimisation de précision"""
        x = np.exp(u)
        y = np.exp(v)

        total = mp.mpf(0)
        n = 1
        max_terms = 15000
        tolerance = mp.mpf(1e-25)

        while n <= max_terms:
            term = mp.exp(-mp.pi * n**2 * (x + y)) * mp.cos(2 * mp.pi * n**2 * mp.sqrt(x * y))
            total += term
            if mp.fabs(term) < tolerance:
                break
            n += 1

        return total / mp.sqrt(x * y)

    def build_matrix(self):
        """Construction de la matrice avec gestion de mémoire optimisée"""
        print(f"Construction de la matrice {self.N}x{self.N}...")
        start_time = time.time()

        K_mat = np.zeros((self.N, self.N))

        # Calcul par blocs pour optimiser la mémoire
        block_size = 200
        for i_block in range(0, self.N, block_size):
            for j_block in range(0, self.N, block_size):
                i_end = min(i_block + block_size, self.N)
                j_end = min(j_block + block_size, self.N)

                for i in range(i_block, i_end):
                    for j in range(j_block, j_end):
                        if j >= i:  # Utiliser la symétrie
                            k_val = float(self.kernel_function(self.nodes[i], self.nodes[j]))
                            weight_factor = self.weights[i] * self.weights[j] / (self.x[i] * self.x[j])
                            K_mat[i, j] = k_val * weight_factor
                            if i != j:
                                K_mat[j, i] = K_mat[i, j]

                print(f"Bloc ({i_block}-{i_end}, {j_block}-{j_end}) complété")

        self.matrix = K_mat
        self.build_time = time.time() - start_time
        print(f"Temps de construction: {self.build_time:.2f} secondes")
        return K_mat

    def compute_eigenvalues(self, n_eig=100):
        """Calcul des valeurs propres avec vérification de stabilité"""
        print(f"Calcul des {n_eig} plus petites valeurs propres...")
        start_time = time.time()

        # Vérification de la symétrie
        symmetry_error = np.max(np.abs(self.matrix - self.matrix.T))
        print(f"Erreur de symétrie: {symmetry_error:.2e}")

        # Calcul des valeurs propres
        self.eigenvalues = la.eigh(self.matrix, eigvals_only=True, subset_by_index=(0, n_eig-1))

        self.eig_time = time.time() - start_time
        print(f"Temps de calcul des valeurs propres: {self.eig_time:.2f} secondes")
        return self.eigenvalues

    def get_riemann_zeros(self, n_zeros):
        """Obtention des zéros de Riemann de référence"""
        zeros = []
        for n in range(1, n_zeros + 1):
            zero = mp.zetazero(n)
            zeros.append(float(zero.imag))
        return zeros

    def comprehensive_analysis(self):
        """Analyse complète avec tous les tests statistiques"""
        n_zeros = len(self.eigenvalues)
        riemann_zeros = self.get_riemann_zeros(n_zeros)

        # Calcul des erreurs
        absolute_errors = np.abs(self.eigenvalues - riemann_zeros)
        relative_errors = absolute_errors / riemann_zeros

        # Analyse des espacements
        sorted_eig = np.sort(self.eigenvalues)
        spacings = np.diff(sorted_eig)
        normalized_spacings = spacings / np.mean(spacings)

        # Test de Kolmogorov-Smirnov avec distribution GUE
        ks_stat, p_value = kstest(normalized_spacings, 'gamma', args=(2, 0, 0.5))

        # Test de normalité des erreurs
        from scipy.stats import shapiro
        shapiro_stat, shapiro_p = shapiro(relative_errors)

        # Analyse de corrélation
        correlation = np.corrcoef(self.eigenvalues, riemann_zeros)[0, 1]

        return {
            'parameters': {
                'U': self.U,
                'N': self.N,
                'precision': self.precision,
                'build_time': self.build_time,
                'eig_time': self.eig_time
            },
            'eigenvalues': self.eigenvalues.tolist(),
            'riemann_zeros': riemann_zeros,
            'absolute_errors': absolute_errors.tolist(),
            'relative_errors': relative_errors.tolist(),
            'mean_absolute_error': np.mean(absolute_errors),
            'max_absolute_error': np.max(absolute_errors),
            'mean_relative_error': np.mean(relative_errors),
            'max_relative_error': np.max(relative_errors),
            'spacings': normalized_spacings.tolist(),
            'ks_test_pvalue': p_value,
            'shapiro_test_pvalue': shapiro_p,
            'correlation_coefficient': correlation,
            'symmetry_error': symmetry_error
        }

# Exécution de la validation finale
def final_validation():
    print("=" * 70)
    print("VALIDATION FINALE - PARAMÈTRES OPTIMAUX")
    print("=" * 70)

    operator = RiemannOperatorFinal(U=12, N=2000, precision=100)
    operator.build_matrix()
    eigenvalues = operator.compute_eigenvalues(100)
    results = operator.comprehensive_analysis()

    # Sauvegarde des résultats
    with open('final_validation_results.json', 'w') as f:
        json.dump(results, f, indent=2)

    # Sauvegarde de la matrice pour analyse future
    np.save('riemann_operator_matrix.npy', operator.matrix)

    return results

if __name__ == '__main__':
    final_results = final_validation()

VALIDATION FINALE - PARAMÈTRES OPTIMAUX
Construction de la matrice 2000x2000...
